# Introduction to the Linux command line

### Data-oriented Programming Paradigms,  2021 WS
10/12/2021

Gábor Recski

There's a huge amount of documentation on Linux programming (start e.g. [here](https://tldp.org/index.html)), this is an introduction to only a very small and specific subset.

The commands below are typically run in a terminal, which is actually the interpreter of a [shell](), such as [bash](), the default shell on most Linux distributions.

To run the commands inside this notebook, you need to install the bash kernel for jupyter, e.g. like this:
```
pip install bash-kernel
python -m bash_kernel.install
```

In [1]:
export LC_ALL=C

The first command sets an __environment variable__ to control the system __locale__ for this session, this ensures that your language and region settings do not interfere with how some tools such as __sort__ work in the examples below. __You need not understand this for now__, but these are still topics worth getting familiar with:
- [locale](https://en.wikipedia.org/wiki/Locale_(computer_software))
- [environment variables](https://en.wikipedia.org/wiki/Environment_variable)
- [sorting and locale](https://stackoverflow.com/questions/28881/why-doesnt-sort-sort-the-same-on-every-machine).

Let's look at the contents of the data directory

In [2]:
ls -l data

total 9056
-rw-rw-r-- 1 eszter eszter  597304 Oct 18 10:24 1984.txt
-rw-rw-r-- 1 eszter eszter  150503 Oct 18 10:24 alice.txt
-rw-rw-r-- 1 eszter eszter  163327 Oct 18 10:24 alice_de.txt
-rw-rw-r-- 1 eszter eszter  157697 Oct 18 10:24 alice_tok.txt
-rw-rw-r-- 1 eszter eszter    3355 Oct 18 10:24 ent_freqs.txt
-rw-rw-r-- 1 eszter eszter     936 Oct 18 10:24 stopwords.txt
-rw-rw-r-- 1 eszter eszter 8188354 Oct 18 10:24 ta_restaurants_31EU.tsv


`ls` is the name of a program for listing directory contents, `-l` is an **option**. Most programs used in this notebook have dozens of options and you shouldn't expect to learn about all of them at once.

You can get a summary of what each command does and what paramteres it has using the `man` command

In [3]:
man ls

LS(1)                            User Commands                           LS(1)

NAME
       ls - list directory contents

SYNOPSIS
       ls [OPTION]... [FILE]...

DESCRIPTION
       List  information  about  the FILEs (the current directory by default).
       Sort entries alphabetically if none of -cftuvSUX nor --sort  is  speci-
       fied.

       Mandatory  arguments  to  long  options are mandatory for short options
       too.

       -a, --all
              do not ignore entries starting with .

       -A, --almost-all
              do not list implied . and ..

       --author
              with -l, print the author of each file

       -b, --escape
              print C-style escapes for nongraphic characters

       --block-size=SIZE
              with  -l,  scale  sizes  by  SIZE  when  printing  them;   e.g.,
              '--block-size=M'; see SIZE format below

       -B, --ignore-backups
              do not list implied entries ending with ~

       -c     with -lt: s

For example, this command will list the directory contents by file size and also use human-readable numbers for file size:

In [4]:
ls -lSh data

total 8.9M
-rw-rw-r-- 1 eszter eszter 7.9M Oct 18 10:24 ta_restaurants_31EU.tsv
-rw-rw-r-- 1 eszter eszter 584K Oct 18 10:24 1984.txt
-rw-rw-r-- 1 eszter eszter 160K Oct 18 10:24 alice_de.txt
-rw-rw-r-- 1 eszter eszter 155K Oct 18 10:24 alice_tok.txt
-rw-rw-r-- 1 eszter eszter 147K Oct 18 10:24 alice.txt
-rw-rw-r-- 1 eszter eszter 3.3K Oct 18 10:24 ent_freqs.txt
-rw-rw-r-- 1 eszter eszter  936 Oct 18 10:24 stopwords.txt


### Day-to-day tasks with pipes

##### What jupyter processes am I running?

In [5]:
ps aux | grep `whoami` | grep jupyter

eszter     15175  0.8  0.5 355580 86572 pts/0    Sl+  10:55   0:04 /home/eszter/.pyenv/versions/3.9.6/envs/phd/bin/python3.9 /home/eszter/.pyenv/versions/phd/bin/jupyter-lab
eszter     15318  0.1  0.3 786924 58004 ?        Ssl  10:55   0:00 /home/eszter/.pyenv/versions/3.9.6/envs/phd/bin/python3.9 -m ipykernel_launcher -f /home/eszter/.local/share/jupyter/runtime/kernel-99c7f8eb-a9b7-4a3b-9204-644fe0a4dd57.json
eszter     15321  0.1  0.3 712756 57000 ?        Ssl  10:55   0:00 /home/eszter/.pyenv/versions/phd/bin/python -m bash_kernel -f /home/eszter/.local/share/jupyter/runtime/kernel-bef79d61-4aef-4af5-9da3-07fccd3957e0.json
eszter     15481  0.1  0.3 713012 57612 ?        Ssl  10:56   0:00 /home/eszter/.pyenv/versions/phd/bin/python -m bash_kernel -f /home/eszter/.local/share/jupyter/runtime/kernel-1a5c066b-4094-4817-870b-3c722831f7e3.json
eszter     16359  0.0  0.0   3316   732 pts/3    S+   11:03   0:00 grep --color=auto jupyter


##### What directories are using up all the disk space?

In [6]:
du -h --max-depth=1 | sort -h

48K	./.ipynb_checkpoints
588K	./media
8.9M	./data
9.6M	.


### Simple data processing with pipes

The file in `data/ta_restaurants_31EU.tsv` contains basic information about restaurants in 31 EU cities, from TripAdvisor. It is based on a [Kaggle dataset](https://www.kaggle.com/damienbeneschi/krakow-ta-restaurans-data-raw).

#### What is in the data?

In [7]:
cat data/ta_restaurants_31EU.tsv | head

# created from original CSV at https://www.kaggle.com/damienbeneschi/krakow-ta-restaurans-data-raw
# fields: Name, City, Cuisine Style, Rating, Price Range, Number of Reviews
Martine of Martine's Table	Amsterdam	French|Dutch|European	5.0	$$ - $$$	136.0
De Silveren Spiegel	Amsterdam	Dutch|European|Vegetarian Friendly|Gluten Free Options	4.5	$$$$	812.0
La Rive	Amsterdam	Mediterranean|French|International|European|Vegetarian Friendly|Vegan Options	4.5	$$$$	567.0
Vinkeles	Amsterdam	French|European|International|Contemporary|Vegetarian Friendly|Vegan Options|Gluten Free Options	5.0	$$$$	564.0
Librije's Zusje Amsterdam	Amsterdam	Dutch|European|International|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.5	$$$$	316.0
Ciel Bleu Restaurant	Amsterdam	Contemporary|International|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.5	$$$$	745.0
Zaza's	Amsterdam	French|International|Mediterranean|European|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.5	$$ - $$$	1455.0
Blue Pepp

#### How many restaurants? How many in Vienna? How many in each city?

In [8]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | wc -l

125523


In [9]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | grep 'Vienna' | wc -l

3732


In [10]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | cut -f2 | sort | uniq -c | sort -nr

  18211 London
  14874 Paris
   9543 Madrid
   8425 Barcelona
   7078 Berlin
   6687 Milan
   5948 Rome
   4859 Prague
   3986 Lisbon
   3724 Vienna
   3434 Amsterdam
   3204 Brussels
   3131 Hamburg
   2995 Munich
   2930 Lyon
   2705 Stockholm
   2605 Budapest
   2352 Warsaw
   2109 Copenhagen
   2082 Dublin
   1938 Athens
   1865 Edinburgh
   1667 Zurich
   1580 Oporto
   1572 Geneva
   1354 Krakow
   1227 Helsinki
   1213 Oslo
   1067 Bratislava
    657 Luxembourg
    501 Ljubljana


#### What are the top cuisines in the dataset?

In [17]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | cut -f3 | tr '|' '\n' | sort | uniq -c | sort -nr | head

  32359 Vegetarian Friendly
  31350 unknown
  30226 European
  18427 Mediterranean
  17794 Italian
  13009 Vegan Options
  12120 Gluten Free Options
   9689 Bar
   9558 French
   9064 Asian


#### Which are the top-rated restaurants in Vienna?

In [16]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | awk -F$'\t' '$2 == "Vienna"' | sort -t $'\t' -k4 -gr | head

wunderladen modecafe	Vienna	Cafe	5.0	$$ - $$$	13.0
s'Kellerstockl	Vienna	Wine Bar	5.0	$$ - $$$	4.0
restaurant Jiang	Vienna	Chinese	5.0	unknown	
repubblica del vino.vienna	Vienna	Italian	5.0	unknown	
kem’s Bar & Kitchenette	Vienna	Italian|Austrian|International|European|Middle Eastern	5.0	$$ - $$$	4.0
engels- die Bar	Vienna	Bar|Fast Food|Pub	5.0	$	8.0
daily Imbiss	Vienna	Indian|Asian|Vegetarian Friendly	5.0	$	42.0
Zypresse	Vienna	Turkish	5.0	$$ - $$$	11.0
Zuppa	Vienna	International|Mediterranean|Asian	5.0	$	3.0
Zum Suppentopf	Vienna	unknown	5.0	unknown	2.0
sort: write failed: 'standard output': Broken pipe
sort: write error


#### OK, how about top-rated cheap restaurants?

In [ ]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | awk -F$'\t' '$2 == "Vienna"'  | grep -v '\$\$' | sort -t $'\t' -k4 -gr | head

#### How about top-rated, cheap, vegan restaurants?

In [18]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | awk -F$'\t' '$2 == "Vienna"'  |  grep '\$' | grep -v '\$\$' | grep 'Vegan Options' | sort -t $'\t' -k4 -gr | head

Reformhaus Staudigl	Vienna	Healthy|Vegetarian Friendly|Vegan Options	5.0	$	8.0
Dem's Gourmet	Vienna	Indian|Asian|Vegetarian Friendly|Vegan Options|Halal	5.0	$	
Cha No Ma	Vienna	Japanese|Cafe|Asian|Vegetarian Friendly|Vegan Options	5.0	$	33.0
B.K Curry Indian Restaurant	Vienna	Indian|Asian|Vegetarian Friendly|Vegan Options|Halal	5.0	$	
neuDeli	Vienna	Indian|Mediterranean|Delicatessen|Vegetarian Friendly|Vegan Options	4.5	$	13.0
Wurstelstand Leo	Vienna	Street Food|German|Austrian|Fast Food|European|Vegan Options	4.5	$	57.0
Vietnam Bistro	Vienna	Asian|Vietnamese|Vegetarian Friendly|Vegan Options	4.5	$	57.0
Trzesniewski	Vienna	Austrian|European|Vegetarian Friendly|Vegan Options	4.5	$	1076.0
Tata Restaurant	Vienna	Asian|Vietnamese|Fusion|Vegetarian Friendly|Vegan Options	4.5	$	106.0
Shelanu	Vienna	Middle Eastern|Israeli|Vegetarian Friendly|Vegan Options|Kosher	4.5	$	2.0


#### Finally, filter those with less than 100 reviews

In [19]:
cat data/ta_restaurants_31EU.tsv | grep -v '^#' | awk -F$'\t' '$2 == "Vienna"'  |  grep '\$' | grep -v '\$\$' | grep 'Vegan Options' | awk -F$'\t' '$6 >= 100' | sort -t $'\t' -k4 -gr 

Trzesniewski	Vienna	Austrian|European|Vegetarian Friendly|Vegan Options	4.5	$	1076.0
Tata Restaurant	Vienna	Asian|Vietnamese|Fusion|Vegetarian Friendly|Vegan Options	4.5	$	106.0
Schillinger's Swing Kitchen	Vienna	American|Fast Food|European|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.5	$	304.0
Schachtelwirt	Vienna	Austrian|Fast Food|European|Central European|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.5	$	463.0
Kolar	Vienna	European|Vegetarian Friendly|Vegan Options	4.5	$	166.0
Blueorange	Vienna	Cafe|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.5	$	116.0
Nguyen's Pho House	Vienna	Asian|Vietnamese|Soups|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.0	$	204.0
Hungry Guy	Vienna	International|Street Food|Middle Eastern|Vegetarian Friendly|Vegan Options|Gluten Free Options	4.0	$	356.0
Der Wiener Deewan	Vienna	Indian|Asian|Pakistani|Vegetarian Friendly|Vegan Options|Halal	4.0	$	238.0
pizza bizi	Vienna	Italian|Pizza|Fast Food|Vegetarian Friendly|Ve